### In this assignment, I will use the Dev01 environment to demonstrate:

1. Launch VSCode and add a terminal
2. Start/stop Cassandra from the vscode terminal
3. Create a Jupyter Lab Notebook (in vscode) and use Python to accomplish the following (steps 4 through 9):
4. Connect to the Cassandra Database
5. Create a keyspace called m14
6. Create a table in this keyspace that will be used to store inventory information. This will include the     fields, SKU (which is shown for stock keeping unit), name (short name for the product), description (longer description of the product), warehouse_num (the warehouse number where the product is stored)
Populate several rows of data into your inventory table
7. Create an index on warehouse_num. 
8. Search for all products located at a given location (for instance - if you created two rows with location 123, list all products at location 123)

### Importing the necessary packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Creating a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Using session to 'talk' to cassandra

In [3]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


### Creating the keyspace m14 with replication factor 3

In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

### Creating inventory table in the m14 keyspace

In [6]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    SKU TEXT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT, 
    PRIMARY KEY(SKU)
);
""")

### Creating a dataframe to persist the values in inventory dataframe

In [7]:
data = {
    'SKU': ['SKU001', 'SKU002', 'SKU003', 'SKU004', 'SKU005'],
    'name': ['Laptop', 'Monitor', 'Smartphone', 'Tablet', 'Wireless Mouse'],
    'description': [
        'High-performance laptop with Intel processor',
        '27-inch 4K Ultra HD monitor with slim bezels',
        'Latest flagship smartphone with OLED display',
        '10-inch tablet with long battery life',
        'Ergonomic wireless mouse for improved productivity'
    ],
    'warehouse_num': [1, 2, 1, 3, 2]
}

df = pd.DataFrame(data)

In [8]:
print(df)

      SKU            name                                        description  \
0  SKU001          Laptop       High-performance laptop with Intel processor   
1  SKU002         Monitor       27-inch 4K Ultra HD monitor with slim bezels   
2  SKU003      Smartphone       Latest flagship smartphone with OLED display   
3  SKU004          Tablet              10-inch tablet with long battery life   
4  SKU005  Wireless Mouse  Ergonomic wireless mouse for improved producti...   

   warehouse_num  
0              1  
1              2  
2              1  
3              3  
4              2  


In [9]:
for index, row in df.iterrows():
    print(f"SKU = {row.SKU}, product_name ={row.name}, product_desc = {row.description}, warehouse_id = {row.warehouse_num}")

SKU = SKU001, product_name =0, product_desc = High-performance laptop with Intel processor, warehouse_id = 1
SKU = SKU002, product_name =1, product_desc = 27-inch 4K Ultra HD monitor with slim bezels, warehouse_id = 2
SKU = SKU003, product_name =2, product_desc = Latest flagship smartphone with OLED display, warehouse_id = 1
SKU = SKU004, product_name =3, product_desc = 10-inch tablet with long battery life, warehouse_id = 3
SKU = SKU005, product_name =4, product_desc = Ergonomic wireless mouse for improved productivity, warehouse_id = 2


### Inserting the values into the cassandra database

In [10]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ({row.SKU}, {row.name}, {row.description}, {row.warehouse_num});
        """
    )
    session.execute(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)
        VALUES ('{row['SKU']}', '{row['name']}', '{row['description']}', {row['warehouse_num']});
        """
    )


        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (SKU001, 0, High-performance laptop with Intel processor, 1);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (SKU002, 1, 27-inch 4K Ultra HD monitor with slim bezels, 2);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (SKU003, 2, Latest flagship smartphone with OLED display, 1);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (SKU004, 3, 10-inch tablet with long battery life, 3);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (SKU005, 4, Ergonomic wireless mouse for improved productivity, 2);
        


In [11]:
rows = session.execute("select (SKU, name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_id={row[0][3]}")


SKU=SKU005, name=Wireless Mouse, description=Ergonomic wireless mouse for improved productivity, warehouse_id=2
SKU=SKU002, name=Monitor, description=27-inch 4K Ultra HD monitor with slim bezels, warehouse_id=2
SKU=SKU004, name=Tablet, description=10-inch tablet with long battery life, warehouse_id=3
SKU=SKU001, name=Laptop, description=High-performance laptop with Intel processor, warehouse_id=1
SKU=SKU003, name=Smartphone, description=Latest flagship smartphone with OLED display, warehouse_id=1


### Creating an index on warehouse_num

In [12]:
session.execute("CREATE INDEX ON m14.inventory(warehouse_num);")

### Searching for products 

In [13]:

rows = session.execute("SELECT * from m14.inventory where warehouse_num = 2;")
for row in rows:
    print(f"{row[0]}, {row[1]}, {row[2]}, {row[3]}")

SKU005, Ergonomic wireless mouse for improved productivity, Wireless Mouse, 2
SKU002, 27-inch 4K Ultra HD monitor with slim bezels, Monitor, 2
